# 回归模型

检查pycaret的版本，确保可以运行。
- 需要2.0.0及以上

注意2.2.0版本如果用 pip install pycaret安装，catboost、xgboost缺省是不装的。
本例中，安装了全集 pip install pycaret[full]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import numpy as np
import pandas as pd

import pycaret
pycaret.__version__

## 课程内容：

- 导入数据：从PyCaret的datasets中导入数据
- 设置环境：在PyCaret中设置实验并开始构建回归模型
- 创建模型：执行交叉验证和评估回归指标
- 调整模型：自动调整回归模型的超参数（需要人为设定的参数叫做超参数）
- 结果绘图：使用各种绘图，分析模型性能
- 预测模型：对测试集进行预测


## 导入数据
使用数据集："Sarah Gets a Diamond"。数据来源于一个浪漫MBA学生(真的，没开玩笑)为自己的新娘莎拉选择合适的钻石的案例。

数据包含6000条记录。每列的简短描述如下：

    ID：每颗钻石的独一无二的编号
    Carat Weight：重量，公制克拉中钻石的重量。
    Cut：切割工艺，五个值之一(Signature-Ideal, Ideal, Very Good, Good, Fair)
    Color：颜色，六个值之一(D, E, F - Colorless, G, H, I - Near colorless)
    Clarity：净度，七个值之一(F - Flawless, IF - Internally Flawless, VVS1 or VVS2 - Very, Very Slightly Included, or VS1 or VS2 - Very Slightly Included, SI1 - Slightly Included)
    Polish：抛光度，四个值之一(ID - Ideal, EX - Excellent, VG - Very Good, G - Good)
    Symmetry：对称性，四个值之一(ID - Ideal, EX - Excellent, VG - Very Good, G - Good)
    Report：评估钻石质量的分级机构，两个值之一("AGSL", "GIA")
    Price：钻石价格

### 数据预处理
利用get_data()函数下载数据集（需要联网）

然后，我们复习一下上节课的内容：对数据处理的异常值检测。

In [ ]:
from pycaret.datasets import get_data
# import pandas as pd
dataset = get_data('diamond')

# file_name='./diamond.csv'
# dataset=pd.read_csv(file_name)
# dataset

查看数据的形状

In [ ]:
dataset.shape

模型对于文字信息的处理能力没有对数字信息强，所以，对数据的文字部分(具体说Cut,Color,Clarity,Polish,Symmetry,Report六列)，我们选择将其转化为对应数字。同时，数字信息也便于绘图和进行计算。

In [ ]:
for i in dataset.columns:
    if i not in ['Carat Weight', 'Price']:
        dataset[i].replace(dataset[i].unique().tolist(), 
                           np.arange(len(dataset[i].unique().tolist())), 
                           inplace = True)

与上面的dataset对比，观察到，文字内容已经变成数字了

In [ ]:
dataset.head()

最后使用杀手锏pandas_profiling一步看清数据的大致特征：

In [ ]:
import pandas_profiling
pfr = pandas_profiling.ProfileReport(dataset)
pfr.to_file('report.html')
pfr

为了方便使用pycaret进行机器学习，我们选择的数据集非常干净，不存在异常值。但是同学们需要注意，真实情况下，数据集往往是不干净的，面对一份全新的数据，清洗工作不可或缺，而常用的清洗方法我们上节课介绍过，当一份数据集被清洗完成后，我们就可以开始机器学习的旅程啦！

还是复习上节课的内容，我们绘制不同特征的两两散点图，来肉眼观察一下数据间的关联性，同时寻找数据是否存在异常。

In [ ]:
sns.set(context='talk', style='ticks', font_scale=0.98)
sns.pairplot(dataset)
plt.show()

由于数据的离散类型较大，所以绘制的图像看着非常工整，原因也很简单，当横纵坐标都是离散值时，绘制的图像只可能在几个特定位置出现数据点。所以这样的数据其实并不能得到很多信息。

- 不过，从图中还是可以得到一些信息的。
- 观察左下角的price-carat weight散点图，明显看到了数据存在正相关关系，说明钻石的价格与克拉重量有正相关关系。
- 观察左上角的carat weight分布图，发现钻石的重量是存在两个峰值的，就是说钻石存在小钻石类和大钻石类，也比较符合我们的认知：碎钻，整钻。
- 观察右下角的price分布图，发现钻石的分布区间明显在低价格部分，说明贵的钻石虽然有人买，但是大部分人还是穷人啊。

### 数据集划分
为了对未知数据进行预测，从原始数据集中保留了600条记录的样本。将总共6000条数据拆分为5400+600条。

- 模拟现实生活场景，总有些情况是没有遇到过的。

In [ ]:
data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

## 设置环境
使用setup()函数初始化pycaret中的环境。

在pycaret中执行任何其他功能之前，必须先调用setup()。
它具有两个必填参数：

    data：填入一个dataframe
    teraget：填入列名

当执行setup()时，PyCaret的推断算法将根据某些属性自动推断所有功能的数据类型,自动判断通常存在错误的可能。为了解决这个问题，在执行setup()之后，PyCaret将显示一个包含特征及其推断的数据类型的表。

    推断正确：Enter键
    推断错误：quit键

pycaret会自动执行一些预处理任务，对于不同数据类型，处理方式往往不同。

In [ ]:
from pycaret.regression import *
exp_reg101 = setup(data = data, target = 'Price')

成功执行set()后，会打印包含一些重要信息的信息表。为了pycaret的通用性，所以显示了大量信息，但是绝大多数信息并没有在本次任务中使用。

## 创建模型

### 评估性能

在完成set()完成后比较所有模型以评估性能（对所有模型进行训练，因为无法确切判断需要使用哪种模型）

此功能训练模型库中的所有模型，并使用k值交叉验证对模型进行度量评估。

输出数据表显示出各种对模型的评价指标：
- average MAE，平均绝对误差
- MSE，均方误差
- RMSE，均方根误差
- R2，决定系数
- RMSLE，均方根对数误差
- MAPE（默认为10），平均绝对百分误差
- training time，训练时间

这些指标背后都存在着数学意义，每个指标都反映了模型的性能。可以参考[知乎评价指标解释](https://zhuanlan.zhihu.com/p/86120987?from_voters_page=true) 

这里有一个小技巧，除了R2以外的所有指标，数值越小表示模型效果越好,R2(小于1)则是数值越大越好。pycaret默认的是R2为评价模型的首要指标，我们也这样做，对模型评价，R2值接近1，意味着模型效果越好。

In [ ]:
best = compare_models()

一行代码，训练和评估了19个模型！！！

数据表高亮了性能最高的模型。默认情况下，表格使用R2（从最高到最低）进行排序。

## 选择模型训练
model()函数显示了pycaret可以使用的回归模型。

In [ ]:
models()

根据模型评估的性能，以及模型的通用程度，我们选择以下三种模型进行进一步训练：

- CatBoost Regressor(效果最好，但是计算慢)
- Linear Regression(最基础)
- Random Forest（使用普遍）


### AdaBoost Regressor

In [ ]:
ada = create_model('ada')

In [ ]:
print(ada)

### Light Gradient Boosting Machine

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
print(lightgbm)

### Decision Tree

In [ ]:
dt = create_model('dt')

In [ ]:
print(dt)

注意，所有模型的平均分数与 compare_models() 中打印的分数相匹配。 这是因为在 compare_models() 分数网格中打印的指标是所有 CV 折叠的平均分数。 与 compare_models() 类似，如果要将 fold 参数从默认值 10 更改为不同的值，则可以使用 fold 参数。 例如： create_model('dt', fold = 5) 使用 5 折交叉验证创建决策树。

## 调整模型

使用create_model函数创建模型时，是使用默认的超参数来训练模型的。在选定了使用模型后，为了提高精度，我们需要调整超参数，pycaret提供了tune_model函数，一键调整超参数。
### Adaboost

In [ ]:
tuned_ada = tune_model(ada)

### Light Gradient Boosting Machine

In [ ]:
import numpy as np
lgbm_params = {'num_leaves': np.arange(10,200,10),
            'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
            'learning_rate': np.arange(0.1,1,0.1)
            }

In [ ]:
tuned_lightgbm = tune_model(lightgbm, custom_grid = lgbm_params)

In [ ]:
print(tuned_lightgbm)

### Decision Tree

In [ ]:
tuned_dt = tune_model(dt)

## 结果绘图

在模型训练完成后，使用plot_model()函数分析不同方面的性能，例如残差图，预测误差图，特征重要性图等。

### 残差图
残差在数理统计中是指真实值与预测值之间的差。

残差图是指以某种残差为纵坐标，以其他适宜的量为横坐标的散点图。
该残差图的横坐标为预测值，纵坐标为残差，看出残差并没有随着预测范围的改变而出现极大变化，比较均匀地分布在预测价格的横坐标上。这反映了模型在预测上并没有”偏科“，对所有价格的钻石都有比较合理的预测结果。
- 特别需要注意的是，蓝色点为训练集，绿色点为测试集，图中明显看出，蓝色点的残差更加小，这是机器学习结果的常态，即训练集效果优于测试集效果。
- 有兴趣的同学可以思考一下为什么会这样？
- 会不会存在测试集效果和训练集效果一样好的情况？
- 是不是训练的残差越小越好。

这些问题的思考有助于同学们加深对机器学习的理解。

In [ ]:
plot_model(tuned_lightgbm)

### 预测偏差

In [ ]:
plot_model(tuned_lightgbm, plot = 'error')

### 预测重要性图

展示了数据不同特征对预测结果的重要性。由图显示，钻石的克拉重量对钻石价格的重要性最大。

In [ ]:
plot_model(tuned_lightgbm, plot='feature')

## 预测模型

还记得最初留下的600个数据嘛？predict_model()函数将模型在看不见的数据集上进行预测。 

我们将传递data_unseen参数。data_unseen是在任务开始时创建的变量，包含从未暴露给PyCaret的原始数据集的10％（600个样本）。 

In [ ]:
unseen_predictions = predict_model(tuned_lightgbm, data=data_unseen)
unseen_predictions.head(20)

Label列添加到data_unseen表的最后一列，label列是使用Random Forest模型对特征进行预测结果，可以发现label列数值与price列数值是非常接近的。

计算预测值与真实值的R2

In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions.Price, unseen_predictions.Label, 'R2')